In [18]:
# Load the 'Main table' sheet from the Excel supplement and tidy up the header
import pandas as pd

# 1) Read the sheet
raw_df = pd.read_csv('positives_NESdb.csv')

raw_df

,candidates,name,species,uniprotID,start#,sequence,secondary,class,loc_DISO,loc_CDD,beta,refDB
0,cand_D,ADAR1,Homo sapiens,P55265,122,GVDCLSSHFQELSIYQ,CCCCCCCCCCCCCCCC,c1a-5,boundary,boundary|z-alpha;,0.000,NESdb:110
1,cand_D,Cdc7,Homo sapiens,O00311,457,DLRKLCERLRGMDS,CHHHHHHHHCCCCC,c3-4,boundary,boundary|STKc_Cdc7; MID|PKc_like superfamily;,0.000,NESdb:197\nvalidNES:P189
2,cand_D,Cdc7,Homo sapiens,O00311,538,VPDEAYDLLDKLLDLNP,CCCHHHHHHHHHCCCCC,c1c-AT-4,boundary,boundary|PKc_like superfamily;,0.000,NESdb:197\nvalidNES:P189
3,cand_D,CPEB4,Homo sapiens,Q17RY0,379,RTFDMHSLESSLIDIM,CCCCCCCCHHHHHHHH,c1aR-5,DISO,NaN,0.000,NESdb:198
4,cand_D,CPEB4,Homo sapiens,Q17RY0,382,DMHSLESSLIDIMR,CCCCCHHHHHHHHC,c3-4,DISO,NaN,0.000,NESdb:198
...,...,...,...,...,...,...,...,...,...,...,...,...
458,cand_D,Zinc finger protein RFP; TRIM27,Homo sapiens,P14373,196,HEYRLLARLEELDLAI,HHHHHHHHHHHHHHHH,c1a-4,boundary,MID|SMC_N superfamily;,0.000,NESdb:43\nvalidNES:P036
459,cand_D,ZO-2; TJP2,Canis familiaris,Q95168,715,RPVVLFGPIADIALEK,CCEEECCCHHHHHHHH,c1a-4,boundary,MID|GuKc;,0.143,NESdb:203\nvalidNES:P115
460,cand_D,ZO-2; TJP2,Canis familiaris,Q95168,724,ADIALEKLANELPDLF,HHHHHHHHHHHCCCCC,c1aR-5,boundary,MID|GuKc;,0.000,NESdb:203\nvalidNES:P115
461,cand_D,Zyxin,Gallus gallus,Q04584,316,NSPLTMKEVEELELLT,CCCCCCHHHHHHHHHH,c1a-AT-4,DISO,boundary|Atrophin-1 superfamily; boundary|LIM1...,0.000,NESdb:23


In [24]:
cols = ['name', 'species', 'uniprotID', 'start#', 'sequence']   # choose the columns you need  
filtered_df = raw_df[cols].copy()   
filtered_df = filtered_df.rename(columns={'sequence': 'NES sequence'})
filtered_df

,name,species,uniprotID,start#,NES sequence
0,ADAR1,Homo sapiens,P55265,122,GVDCLSSHFQELSIYQ
1,Cdc7,Homo sapiens,O00311,457,DLRKLCERLRGMDS
2,Cdc7,Homo sapiens,O00311,538,VPDEAYDLLDKLLDLNP
3,CPEB4,Homo sapiens,Q17RY0,379,RTFDMHSLESSLIDIM
4,CPEB4,Homo sapiens,Q17RY0,382,DMHSLESSLIDIMR
...,...,...,...,...,...
458,Zinc finger protein RFP; TRIM27,Homo sapiens,P14373,196,HEYRLLARLEELDLAI
459,ZO-2; TJP2,Canis familiaris,Q95168,715,RPVVLFGPIADIALEK
460,ZO-2; TJP2,Canis familiaris,Q95168,724,ADIALEKLANELPDLF
461,Zyxin,Gallus gallus,Q04584,316,NSPLTMKEVEELELLT


In [34]:
import pandas as pd, requests, time  
from tqdm import tqdm  
  
# --- assume filtered_df already exists ------------------------------  
# it contains the column 'uniprotID'  
  
def fetch_uniprot_fasta(uid):  
    """Return the primary sequence for a single UniProt accession or None if unavailable."""  
    if pd.isna(uid):  
        return None  
    url = 'https://rest.uniprot.org/uniprotkb/' + uid + '.fasta'  
    try:  
        r = requests.get(url, headers={'Accept': 'text/plain'}, timeout=10)  
        if r.status_code != 200 or not r.text.startswith('>'):  
            return None                     # no FASTA returned  
        # strip header line and join the remaining lines  
        return ''.join(r.text.split('\n')[1:]).strip()  
    except Exception:  
        return None                         # connectivity or parsing error  
  
# --------------------------------------------------  
# build a lookup once, then map it back to the frame  
# --------------------------------------------------  
unique_ids = filtered_df['uniprotID'].dropna().unique()  
seq_lookup = {}  
  
print('Contacting UniProt …')  
for uid in tqdm(unique_ids):  
    seq_lookup[uid] = fetch_uniprot_fasta(uid)  
    time.sleep(0.3)                         # polite 3–4 requests / sec  
  
# attach the full sequences  
filtered_df['full sequence'] = filtered_df['uniprotID'].map(seq_lookup)  
  
# quick sanity-check  
print(filtered_df[['uniprotID', 'NES sequence', 'full sequence']].head())

Contacting UniProt …


100%|████████████████████████████████████████████████████████████████████████████████| 255/255 [02:43<00:00,  1.56it/s]

  uniprotID       NES sequence  \
0    P55265   GVDCLSSHFQELSIYQ   
1    O00311     DLRKLCERLRGMDS   
2    O00311  VPDEAYDLLDKLLDLNP   
3    Q17RY0   RTFDMHSLESSLIDIM   
4    Q17RY0     DMHSLESSLIDIMR   

                                       full sequence  
0  MNPRQGYSLSGYYTHPFQGYEHRQLRYQQPGPGSSPSSFLLKQIEF...  
1  MEASLGIQMDEPMAFSPQRDRFQAEGSLKKNEQNFKLAGVKKDIEK...  
2  MEASLGIQMDEPMAFSPQRDRFQAEGSLKKNEQNFKLAGVKKDIEK...  
3  MGDYGFGVLVQSNTGNKSAFPVRFHPHLQPPHHHQNATPSPAAFIN...  
4  MGDYGFGVLVQSNTGNKSAFPVRFHPHLQPPHHHQNATPSPAAFIN...  
